In [237]:
import pandas as pd
import ast
from tqdm import tqdm
import dask.dataframe as dd
import sqlite3
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
import plotly.express as px
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

In [8]:
class SQLDataBase:
    def __init__(self, path_to_db, table_name):
        self.db_name = f"{path_to_db}/database.db"
        self.table_name = table_name
        self.conn = sqlite3.connect(self.db_name)

    def append(self, data: pd.DataFrame):
        data.to_sql(self.table_name, con=self.conn, if_exists='append', index=False)

    def close(self):
        self.conn.close()

In [11]:
path_to_folder = Path('./wild') 
path_to_save = Path('./papka')

In [146]:
pd.options.display.max_columns = 50

In [3]:
df = pd.read_csv('./WB_hack_3.csv', header=None)

In [4]:
df.columns =['utc_event_time', 'utc_event_date', 'user_id', 'city_name', 'ecom_event_action', 'ecom_id', 'ecom_brand', 'ecom_variant', 'ecom_currency', 'ecom_price100', 'ecom_qty', 'ecom_grand_total100', 'os_manufacturer', 'device_type', 'traffic_src_kind', 'app_version', 'net_type', 'locale']

In [5]:
df['ecom_event_action'] = df['ecom_event_action'].apply(lambda x: ast.literal_eval(x))

df['ecom_id'] = df['ecom_id'].apply(lambda x: ast.literal_eval(x))

df['ecom_brand'] = df['ecom_brand'].apply(lambda x: ast.literal_eval(x))

df['ecom_variant'] = df['ecom_variant'].apply(lambda x: ast.literal_eval(x))

df['ecom_currency'] = df['ecom_currency'].apply(lambda x: ast.literal_eval(x))

df['ecom_price100'] = df['ecom_price100'].apply(lambda x: ast.literal_eval(x))

df['ecom_qty'] = df['ecom_qty'].apply(lambda x: ast.literal_eval(x))

df['ecom_grand_total100'] = df['ecom_grand_total100'].apply(lambda x: ast.literal_eval(x))

In [6]:
df['lonf'] = 0

for i in range(len(df)):
    if len(df.ecom_event_action[i]) != 1:
        df['lonf'][i] = 1

new_df = df[df['lonf']==1]

new_df.reset_index(drop=True, inplace=True)

new_df.head(1)

<ipython-input-6-9e3f909909eb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lonf'][i] = 1


,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf
0,2021-10-18 07:30:08,2021-10-18,10431773979065916196,Минск,"[begin_checkout, begin_checkout, begin_checkou...","[24E32C81C872785E27CB1F89D4236565, EEF2D5D6555...",[3289197B71E5C3F032F5735C9F02ABE7EFC531C923584...,[6348C49CA6A050B7F813298B656179BDC1609083B3049...,"[RUB, RUB, RUB, RUB, RUB]","[50100, 18100, 16400, 4400, 22000]","[1, 1, 1, 1, 1]","[50100, 18100, 16400, 4400, 22000]",Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1


In [22]:
new_df['user_id'] = new_df['user_id'].apply(lambda x: str(x))

<ipython-input-22-2647e30888ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['user_id'] = new_df['user_id'].apply(lambda x: str(x))


In [23]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79317 entries, 0 to 79316
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   utc_event_time       79317 non-null  object
 1   utc_event_date       79317 non-null  object
 2   user_id              79317 non-null  object
 3   city_name            70815 non-null  object
 4   ecom_event_action    79317 non-null  object
 5   ecom_id              79317 non-null  object
 6   ecom_brand           79317 non-null  object
 7   ecom_variant         79317 non-null  object
 8   ecom_currency        79317 non-null  object
 9   ecom_price100        79317 non-null  object
 10  ecom_qty             79317 non-null  object
 11  ecom_grand_total100  79317 non-null  object
 12  os_manufacturer      79317 non-null  object
 13  device_type          79317 non-null  object
 14  traffic_src_kind     79317 non-null  object
 15  app_version          79317 non-null  object
 16  net_

In [24]:
def add_actions(df):
    db = SQLDataBase(path_to_save, 'table')
    for i in tqdm(range(len(df))):
        ddf_s = dd.from_pandas(pd.DataFrame(columns=df.columns, index = range(len(new_df['ecom_event_action'][i]))), npartitions=1)
        ddf_s['utc_event_time'] = new_df['utc_event_time'][i]
        ddf_s['utc_event_date'] = new_df['utc_event_date'][i]
        ddf_s['user_id'] = new_df['user_id'][i]
        ddf_s['city_name'] = new_df['city_name'][i]
        ddf_s['ecom_event_action'] = pd.Series(new_df['ecom_event_action'][i])
        ddf_s['ecom_id'] = pd.Series(new_df['ecom_id'][i])
        ddf_s['ecom_brand'] = pd.Series(new_df['ecom_brand'][i])
        ddf_s['ecom_variant'] = pd.Series(new_df['ecom_variant'][i])
        ddf_s['ecom_currency'] = pd.Series(new_df['ecom_currency'][i])
        ddf_s['ecom_price100'] = pd.Series(new_df['ecom_price100'][i])
        ddf_s['ecom_grand_total100'] = pd.Series(new_df['ecom_grand_total100'][i])
        ddf_s['os_manufacturer'] = new_df['os_manufacturer'][i]
        ddf_s['os_manufacturer'] = new_df['os_manufacturer'][i]
        ddf_s['device_type'] = new_df['device_type'][i]
        ddf_s['traffic_src_kind'] = new_df['traffic_src_kind'][i]
        ddf_s['app_version'] = new_df['app_version'][i]
        ddf_s['net_type'] = new_df['net_type'][i]
        ddf_s['locale'] = new_df['locale'][i]
        ddf_s['lonf'] = new_df['lonf'][i]
        db.append(ddf_s.compute())
    db.close()

    os.system(f"""sqlite3 -header -csv {path_to_save / 'database.db'} "select * from 'table';" > {path_to_save / path_to_folder.name}.csv""")
    return 'DONE'

In [26]:
add_actions(new_df)

100%|██████████████████████████████████████████████████████████████████████████| 79317/79317 [1:54:48<00:00, 11.51it/s]


'DONE'

In [2]:
df_to_add = pd.read_csv('./papka/wild.csv')

In [3]:
df_to_add.drop_duplicates(inplace=True)

In [4]:
df_to_add['ecom_qty'] = df_to_add['ecom_grand_total100'] / df_to_add['ecom_price100']

In [5]:
df_to_add

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf
0,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,24E32C81C872785E27CB1F89D4236565,3289197B71E5C3F032F5735C9F02ABE7EFC531C9235846...,6348C49CA6A050B7F813298B656179BDC1609083B3049E...,RUB,50100,1.0,50100,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
1,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,EEF2D5D6555A6E8293D2F160F3AFAB7A,911ECE2EEA64EE2B1C99B21DA2525328,43FCC4D7C1455DAAD680C4F142A539FB,RUB,18100,1.0,18100,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
2,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,2BF012C5ED6812FA92625638FF03A8E7,7156FA2713A0C80CD1AD0DAAC8203CD4,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,16400,1.0,16400,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
3,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,21BC2D15370D174A820F6CD1D773A0DB,F6BEB36A04259A8EFDBE24A77C6BF0F0,3B9A6121A509CBADC049AEABD3ED0223,RUB,4400,1.0,4400,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
4,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,73448589845BAD0A7DA979CEFDCA4326,F20F2ACE7C7DD17D86BBFB4F53A5744F,3B9A6121A509CBADC049AEABD3ED0223,RUB,22000,1.0,22000,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822633,2021-10-10 16:55:41,2021-10-10,1.270048e+19,Москва,begin_checkout,4471C612A8DC0B56607EBED79E3FA3B9,7F39173DE4DD955DE98C28BEBABF1F76,0390E36F0FEB3BECB16F22362259FF43,RUB,52700,3.0,158100,HUAWEI,smartphone,mobile,4.3.3000,Wi-Fi,ru,1
822634,2021-10-10 16:56:19,2021-10-10,1.270048e+19,Москва,begin_checkout,9EC302050B63C90533F9FFBCAC489DD8,A045311D713A9F98D5DBEE79D58CF7F806A50A1B0F92C3...,77544CB78DD75439AC036C9EA66445A6,RUB,17400,1.0,17400,HUAWEI,smartphone,mobile,4.3.3000,Wi-Fi,ru,1
822635,2021-10-10 16:56:19,2021-10-10,1.270048e+19,Москва,begin_checkout,4471C612A8DC0B56607EBED79E3FA3B9,7F39173DE4DD955DE98C28BEBABF1F76,0390E36F0FEB3BECB16F22362259FF43,RUB,52700,4.0,210800,HUAWEI,smartphone,mobile,4.3.3000,Wi-Fi,ru,1
822636,2021-10-10 16:56:33,2021-10-10,1.270048e+19,Москва,purchase,9EC302050B63C90533F9FFBCAC489DD8,A045311D713A9F98D5DBEE79D58CF7F806A50A1B0F92C3...,77544CB78DD75439AC036C9EA66445A6,RUB,17400,1.0,17400,HUAWEI,smartphone,mobile,4.3.3000,Wi-Fi,ru,1


In [6]:
df_full=pd.concat([df_to_add, pd.read_csv('./one_event.csv')])

In [37]:
df_full.to_csv('./wild_df.csv', index=False)

In [64]:
df_to_add.groupby(['utc_event_date', 'user_id']).agg({'city_name':'nunique'}).sort_values('city_name').reset_index()['user_id'][38555].astype(float)

1.3452540509857e+19

In [34]:
df_full = dd.DataFrame(dsk={}, name='', meta=pd.DataFrame(columns=df.columns), divisions=range(0))

In [9]:
df_full

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf
0,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,24E32C81C872785E27CB1F89D4236565,3289197B71E5C3F032F5735C9F02ABE7EFC531C9235846...,6348C49CA6A050B7F813298B656179BDC1609083B3049E...,RUB,50100,1.0,50100,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
1,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,EEF2D5D6555A6E8293D2F160F3AFAB7A,911ECE2EEA64EE2B1C99B21DA2525328,43FCC4D7C1455DAAD680C4F142A539FB,RUB,18100,1.0,18100,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
2,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,2BF012C5ED6812FA92625638FF03A8E7,7156FA2713A0C80CD1AD0DAAC8203CD4,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,16400,1.0,16400,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
3,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,21BC2D15370D174A820F6CD1D773A0DB,F6BEB36A04259A8EFDBE24A77C6BF0F0,3B9A6121A509CBADC049AEABD3ED0223,RUB,4400,1.0,4400,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
4,2021-10-18 07:30:08,2021-10-18,1.043177e+19,Минск,begin_checkout,73448589845BAD0A7DA979CEFDCA4326,F20F2ACE7C7DD17D86BBFB4F53A5744F,3B9A6121A509CBADC049AEABD3ED0223,RUB,22000,1.0,22000,Xiaomi,smartphone,mobile,4.2.9000,4G,ru,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634024,2021-10-10 11:13:11,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,0
2634025,2021-10-10 12:10:52,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,0
2634026,2021-10-10 11:39:28,2021-10-10,1.098597e+19,Москва,view_item,BDBD23C5359EF7B3F85763C032892B85,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,2309200,1.0,2309200,HUAWEI,smartphone,mobile,4.2.5000-huawei,4G,ru,0
2634027,2021-10-10 11:02:27,2021-10-10,1.240916e+19,Москва,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74900,1.0,74900,samsung,smartphone,mobile,4.3.1003,Wi-Fi,ru,0


In [10]:
df_full.sort_values('city_name')

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf
1532413,2021-10-04 11:36:54,2021-10-04,1.104859e+19,Arys’,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,67600,1.0,67600,samsung,smartphone,mobile,4.3.1003,Wi-Fi,ru,0
532727,2021-10-19 14:53:43,2021-10-19,1.111995e+19,Berëza,purchase,E08C127AE2953899C3EAA2B4478D5BF5,309A2A50C4AC2ABB01CA7B5AA607E5C8,3B9A6121A509CBADC049AEABD3ED0223,RUB,10100,1.0,10100,HUAWEI,smartphone,mobile,4.3.4000,4G,ru,1
565651,2021-09-03 10:40:18,2021-09-03,1.182014e+19,Berëza,begin_checkout,8151794D79C225943E169F791BC7A66C,E5E9CA1C614AF7451563D2A834B7AB5A,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,74400,1.0,74400,samsung,smartphone,mobile,4.2.3000,3G,ru,1
565650,2021-09-03 10:40:18,2021-09-03,1.182014e+19,Berëza,begin_checkout,D0DAD9A15B72E9919348B2804796A510,6B7580F9694CFF535D732C54FE67D505,ECA39849CD4A837A50F1095962623A6B,RUB,54100,1.0,54100,samsung,smartphone,mobile,4.2.3000,3G,ru,1
565649,2021-09-03 10:40:18,2021-09-03,1.182014e+19,Berëza,begin_checkout,1DC449D0C09884C2C534F162132B3910,572D716F747ED00FA3F5DA9C94D8CA8C,43FCC4D7C1455DAAD680C4F142A539FB,RUB,176600,1.0,176600,samsung,smartphone,mobile,4.2.3000,3G,ru,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633976,2021-10-10 10:14:07,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633977,2021-10-10 10:14:14,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633978,2021-10-10 10:14:25,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633979,2021-10-10 12:27:40,2021-10-10,1.380813e+19,NaN,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74900,1.0,74900,HUAWEI,smartphone,mobile,4.2.7000,3G,ru,0


In [12]:
df_full[df_full['city_name'].isna()]

,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf
93,2021-10-30 13:07:10,2021-10-30,1.290171e+19,NaN,begin_checkout,A6FC1D41C2A36CD6ED1428BC40991F3A,CBC88053A1619544EB9FEDAD72FAF8E6DF71A09C7DDFDE...,77544CB78DD75439AC036C9EA66445A6,RUB,18000,1.0,18000,HUAWEI,smartphone,mobile,4.3.2000,3G,ru,1
94,2021-10-30 13:07:10,2021-10-30,1.290171e+19,NaN,begin_checkout,05CA6778FB07B65FF4326D32D088E770,48E72A507D6BCE8447D89BA9E1314A02,5E5CCD7A1417CCEAA63B8318CD67798F,RUB,116900,1.0,116900,HUAWEI,smartphone,mobile,4.3.2000,3G,ru,1
95,2021-10-30 13:07:10,2021-10-30,1.290171e+19,NaN,begin_checkout,28524FBED57B7F0C1C3C0EA673FD4244,9EE2DB887D373AAAA93934613D395B2F,3B9A6121A509CBADC049AEABD3ED0223,RUB,16000,1.0,16000,HUAWEI,smartphone,mobile,4.3.2000,3G,ru,1
96,2021-10-30 13:07:10,2021-10-30,1.290171e+19,NaN,begin_checkout,1E588ADCF872E16869EF334A6A03FE8C,9EE2DB887D373AAAA93934613D395B2F,3B9A6121A509CBADC049AEABD3ED0223,RUB,28100,1.0,28100,HUAWEI,smartphone,mobile,4.3.2000,3G,ru,1
97,2021-10-30 13:07:10,2021-10-30,1.290171e+19,NaN,begin_checkout,01E7D2DCFBE2CB7DFE9F098A9211BEE9,1D1180B11069654926796AA246E763ED,3B9A6121A509CBADC049AEABD3ED0223,RUB,24300,1.0,24300,HUAWEI,smartphone,mobile,4.3.2000,3G,ru,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633976,2021-10-10 10:14:07,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633977,2021-10-10 10:14:14,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633978,2021-10-10 10:14:25,2021-10-10,1.112732e+19,NaN,view_item,6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,170600,1.0,170600,Sony,smartphone,mobile,4.2.9000,3G,ru,0
2633979,2021-10-10 12:27:40,2021-10-10,1.380813e+19,NaN,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74900,1.0,74900,HUAWEI,smartphone,mobile,4.2.7000,3G,ru,0


In [14]:
df_full.groupby('city_name').agg({'user_id': 'nunique'}).sort_values('user_id')

,user_id
city_name,
Arys’,1
Бурбулатово,1
Нижнеусцелемово,1
Новая Каховка,1
Нови-Сад,1
...,...
Казань,41256
Краснодар,54740
Екатеринбург,63894


In [15]:
df_full['utc_event_time'] = df_full['utc_event_time'].apply(lambda x: pd.to_datetime(x))

In [16]:
df_full['utc_event_date'] = df_full['utc_event_date'].apply(lambda x: pd.to_datetime(x))

In [109]:
df_cities = df_full[(df_full.city_name == 'Москва') | (df_full.city_name == 'Казань') | (df_full.city_name == 'Краснодар') | (df_full.city_name == 'Екатеринбург') | (df_full.city_name == 'Санкт-Петербург')]

In [111]:
df_cities.ecom_id.nunique()

112998

In [133]:
df_cities.reset_index(inplace=True)

In [123]:
scaler = StandardScaler()

In [140]:
df_cities['ecom_id_int'] = df_cities['ecom_id'].apply(lambda x: int.from_bytes(x.encode('utf8'), 'little'))

<ipython-input-140-55efc49f3a90>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [380]:
km = KMeans(n_clusters=3)

In [382]:
df_cities

,index,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf,cluster,ecom_id_int,month,ecom_brand_int
0,5,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,6EB3A8836EBDF486669F65CA40FCB6D9,12D89E762536342E35B5F410066B0B45,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,38900,1.0,38900,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2590235245039511832665998916842369895663447726...,10,3175601312969645867688948142551502234471299204...
1,6,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,C0C9F2DC21904776545FFBB2FC3D646C,A1311DE377A4D46E27D2AE29A4755DB8,3B9A6121A509CBADC049AEABD3ED0223,RUB,15400,1.0,15400,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,1,2,3040073095040635803230787599205470328296931970...,10,2315664401956998774432220060660872264289697689...
2,7,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,6614542C68CA1F63F168B25FC75D6001,BF6C2D5F4A28EE8D6451DEC5A5435258,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,23600,1.0,23600,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2224847098653430902403536330179467085005120641...,10,3175601312969645867688948142551502234471299204...
3,8,2021-10-21 14:09:28,2021-10-21,9.965105e+18,Москва,purchase,6EB3A8836EBDF486669F65CA40FCB6D9,12D89E762536342E35B5F410066B0B45,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,38900,1.0,38900,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2590235245039511832665998916842369895663447726...,10,3175601312969645867688948142551502234471299204...
4,9,2021-10-21 14:09:28,2021-10-21,9.965105e+18,Москва,purchase,C0C9F2DC21904776545FFBB2FC3D646C,A1311DE377A4D46E27D2AE29A4755DB8,3B9A6121A509CBADC049AEABD3ED0223,RUB,15400,1.0,15400,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,1,2,3040073095040635803230787599205470328296931970...,10,2315664401956998774432220060660872264289697689...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219402,2634024,2021-10-10 11:13:11,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,1,2542177667976385554899154319750387031182178887...,10,2997672208113748720453844311523796140687624925...
1219403,2634025,2021-10-10 12:10:52,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,1,2542177667976385554899154319750387031182178887...,10,2997672208113748720453844311523796140687624925...
1219404,2634026,2021-10-10 11:39:28,2021-10-10,1.098597e+19,Москва,view_item,BDBD23C5359EF7B3F85763C032892B85,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,2309200,1.0,2309200,HUAWEI,smartphone,mobile,4.2.5000-huawei,4G,ru,1,1,2407198127982284799811285661902920498956752060...,10,2315664401956998774432220060660872264289697689...
1219405,2634027,2021-10-10 11:02:27,2021-10-10,1.240916e+19,Москва,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74900,1.0,74900,samsung,smartphone,mobile,4.3.1003,Wi-Fi,ru,1,0,2589879117644869679362969964144590036682346368...,10,2315664401956998774432220060660872264289697689...


In [385]:
X_sc = scaler.fit_transform(df_cities[['ecom_id_int', 'ecom_price100']])

In [195]:
labels = km.fit_predict(X_sc)

In [196]:
df_cities['cluster'] = labels

<ipython-input-196-04a0c74274c3>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [198]:
df_cities

,index,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf,cluster,ecom_id_int,month
0,5,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,6EB3A8836EBDF486669F65CA40FCB6D9,12D89E762536342E35B5F410066B0B45,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,38900,1.0,38900,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2590235245039511832665998916842369895663447726...,10
1,6,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,C0C9F2DC21904776545FFBB2FC3D646C,A1311DE377A4D46E27D2AE29A4755DB8,3B9A6121A509CBADC049AEABD3ED0223,RUB,15400,1.0,15400,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,1,2,3040073095040635803230787599205470328296931970...,10
2,7,2021-10-21 14:09:23,2021-10-21,9.965105e+18,Москва,begin_checkout,6614542C68CA1F63F168B25FC75D6001,BF6C2D5F4A28EE8D6451DEC5A5435258,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,23600,1.0,23600,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2224847098653430902403536330179467085005120641...,10
3,8,2021-10-21 14:09:28,2021-10-21,9.965105e+18,Москва,purchase,6EB3A8836EBDF486669F65CA40FCB6D9,12D89E762536342E35B5F410066B0B45,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,38900,1.0,38900,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,0,0,2590235245039511832665998916842369895663447726...,10
4,9,2021-10-21 14:09:28,2021-10-21,9.965105e+18,Москва,purchase,C0C9F2DC21904776545FFBB2FC3D646C,A1311DE377A4D46E27D2AE29A4755DB8,3B9A6121A509CBADC049AEABD3ED0223,RUB,15400,1.0,15400,Xiaomi,smartphone,mobile,4.3.5000,Wi-Fi,ru,1,2,3040073095040635803230787599205470328296931970...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219402,2634024,2021-10-10 11:13:11,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,1,2542177667976385554899154319750387031182178887...,10
1219403,2634025,2021-10-10 12:10:52,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,1,2542177667976385554899154319750387031182178887...,10
1219404,2634026,2021-10-10 11:39:28,2021-10-10,1.098597e+19,Москва,view_item,BDBD23C5359EF7B3F85763C032892B85,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,2309200,1.0,2309200,HUAWEI,smartphone,mobile,4.2.5000-huawei,4G,ru,1,1,2407198127982284799811285661902920498956752060...,10
1219405,2634027,2021-10-10 11:02:27,2021-10-10,1.240916e+19,Москва,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,74900,1.0,74900,samsung,smartphone,mobile,4.3.1003,Wi-Fi,ru,1,0,2589879117644869679362969964144590036682346368...,10


In [163]:
df_cities['month'] = df_cities['utc_event_date'].apply(lambda x: x.month)

<ipython-input-163-6f239726eda7>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [199]:
df_cities.to_csv('./wildhack/input/df_short.csv')

In [177]:
products = ['7D9A98C6E8BF033613B74252D9D24B33',
            '9EC302050B63C90533F9FFBCAC489DD8',
            'AC688D3B8E8D812E2B49496BE59286E3',
            '0F49D92CEA8C80166B0605ED3D1A8E5D',
            '6050B6B1F05ABDB4B33DA4BCEB46DE9B',
            '5D199BE9EFD733A097DF8E08C9B85924']

In [180]:
df_seller = pd.DataFrame([df_cities.iloc[i] for i in range(len(df_cities)) if df_cities['ecom_id'][i] in products])

In [181]:
df_seller

,index,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale,lonf,cluster,ecom_id_int,month
23,56,2021-10-14 18:00:31,2021-10-14,1.054565e+19,Санкт-Петербург,purchase,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,7948A713612D01DC921228E236CAA06A,RUB,75700,1.0,75700,samsung,smartphone,mobile,4.3.1003,3G,ru,1,2,2589879117644869679362969964144590036682346368...,10
25,67,2021-10-14 17:53:56,2021-10-14,1.085119e+19,Москва,begin_checkout,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,1175200,1.0,1175200,HUAWEI,smartphone,mobile,4.3.4000,Wi-Fi,ru,1,0,2589892179748905514837945898544559589541058158...,10
32,74,2021-10-14 17:54:00,2021-10-14,1.085119e+19,Москва,purchase,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,1175200,1.0,1175200,HUAWEI,smartphone,mobile,4.3.4000,Wi-Fi,ru,1,0,2589892179748905514837945898544559589541058158...,10
39,81,2021-10-11 09:25:46,2021-10-11,1.147699e+19,Краснодар,purchase,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,ECA39849CD4A837A50F1095962623A6B,RUB,126600,1.0,126600,HUAWEI,smartphone,mobile,4.3.3000,4G,ru,0,2,2315852140052007846133637257286800460995660028...,10
56,153,2021-09-26 20:25:27,2021-09-26,1.227395e+19,Москва,purchase,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,1065500,1.0,1065500,Xiaomi,smartphone,mobile,4.2.9000,Wi-Fi,ru,1,0,2589892179748905514837945898544559589541058158...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219401,2634023,2021-10-10 11:10:33,2021-10-10,1.333123e+19,Екатеринбург,add_to_cart,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,0,2542177667976385554899154319750387031182178887...,10
1219402,2634024,2021-10-10 11:13:11,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,0,2542177667976385554899154319750387031182178887...,10
1219403,2634025,2021-10-10 12:10:52,2021-10-10,1.333123e+19,Екатеринбург,begin_checkout,8360E6D158AEBD26BBBE835C38F88748,54C53BB4CEB1938D929A930E4FFC4BE5,43FCC4D7C1455DAAD680C4F142A539FB,RUB,1382400,1.0,1382400,samsung,smartphone,mobile,4.2.9000,2G,ru,1,0,2542177667976385554899154319750387031182178887...,10
1219404,2634026,2021-10-10 11:39:28,2021-10-10,1.098597e+19,Москва,view_item,BDBD23C5359EF7B3F85763C032892B85,0B1BC368C77D7A9B58F26B087958C231,3B9A6121A509CBADC049AEABD3ED0223,RUB,2309200,1.0,2309200,HUAWEI,smartphone,mobile,4.2.5000-huawei,4G,ru,1,3,2407198127982284799811285661902920498956752060...,10


In [271]:
df_seller.to_csv('./wildhack/input/df_seller.csv')